In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

house_df = pd.read_csv('kc_house_data.csv')

In [ ]:
# something is bugged
# this can't be together with imports: https://github.com/jupyter/notebook/issues/3385
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300.0 # default: 72.0
mpl.rcParams['figure.figsize'] = [8., 4.5] # default: [6., 4.]

## 1. Linear Regression. A bit of formalism

Let's have a data consisting of $n$ samples. These samples are $(m+1)$ - dimensional vectors, where first $m$ dimensions are called features (explanatory variables) and will be used to predict the last dimension which is called regressand (dependent variable). 

We will have therefore a $n \times m$ matrix **$X$** (called feature matrix) and vector **$y$** of lenght $n$. 

In our simple example, living squarefeet is a feature, house's price is regressand and the data consists of $100$ samples. It forms $100 \times 1$ feature matrix **$X$** and vector **$y$** of length $100$.

In [ ]:
X = house_df.sqft_living.values.reshape(-1,1)[:100]
y = house_df.price.values.reshape(-1,1)[:100]
print(np.shape(X))
print(np.shape(y))

Linear regression approach tries to find a vector **$b$** which minimizes the cost function 

$$f_{y}(b) = \dfrac{|Xb - y|^2}{n}, $$

where **$|\cdot|$** is euclidean norm and **$Xb$** is simple matrix multiplication (vector is also a matrix). 

Geometrically (and roughly) speaking, we are determining a line which minimizes the cumulative distance to all the points. 

When such a vector **$b$** is found, we can predict values **$y$** for given features **$X$** by calculating **$Xb$**. We have therefore 

$$y_{pred} = Xb, \\ f_{y}(b) = \dfrac{|y_{pred} - y|^2}{n}. $$

## Exercise 1
a) Create functions *predict(b, X)* and *cost(y_pred, y_true)* implementing theory above.  All inputs should be numpy arrays (take care of dimension scalability!)  
b) For our data **$X,y$**, plot cost as the function of **$b$** (which is one dimensional vector). Plot the line with best slope among  evaluated with *cost* function.

In [ ]:
#a)
def predict(b, X):
    return np.dot(X, b)

def cost(y_pred, y_true):
   #return np.linalg.norm(y_pred - y_true) ** 2 / len(y_pred)
    return np.sum((y_pred - y_true) ** 2) / len(y_pred)

#short test:
test_b = np.array([[1],[1],[1]])
test_X = np.array([[1,2,3],[2,5,4],[3,4,5],[4,5,7]])
test_y = np.array([[1],[2],[3],[4]])

prediction = predict(test_b, test_X)
print(prediction)
print(cost(prediction, test_y))

In [ ]:
#b)
fy = lambda b: cost(predict(b, X), y)

t = np.arange(-1000, 1000, 0.25)
ft = list(map(fy, t))
plt.plot(t, ft)
plt.show()

plt.plot(t[4800:5200], ft[4800:5200])
plt.show()

from scipy.optimize import fmin
min_fy = fmin(fy, 250, disp=False)[0]

from scipy.optimize import minimize
min_fy2 = minimize(fy, 200).x[0]

min_fy3 = t[np.argmin([fy(b) for b in t])]

plt.scatter(X, y, color='red')
plt.plot(X, predict(min_fy, X), label=f'bias = {min_fy}, cost = {cost(predict(min_fy, X), y)}')
plt.plot(X, predict(min_fy2, X), label=f'bias = {min_fy2}, cost = {cost(predict(min_fy2, X), y)}')
plt.plot(X, predict(min_fy3, X), label=f'bias = {min_fy3}, cost = {cost(predict(min_fy3, X), y)}')
plt.legend()
plt.gca().set_xlim([0,6000])
plt.show()


# 2. Bias

The line above has limitation of crossing point (0,0). As on our data it doesn't seem harmful, let us check how our model would perform, if all $y$ were shifted by some constant. The interpretation is as follows: the goverment has imposed a new tax in the set amount of 1 million dollar on buying new houses.

As we can see, for the best slope the error has grown almost by the factor of four. In addition we clearly see, that what limits us is indeed the constraint of all lines crossing point $(0,0)$.   

There are two basic solutions of this problem. One of them is centring of the data, that is substracting overall mean from all the values of given feature (or regressand).  

The other is adding bias term, which in this context (different than context discussed during the lectures!) may be understood as constant term in line equation. We simulate adding constant term to result of each prediction by appending artificial feature consisting of ones to **$X$** matrix and additional term to **$b$** vector.

## Exercise 2
a) Add bias column to the **$X$** matrix.  
b) Make the code below compatible with functions that you've created so far.

In [ ]:
#a)
if X.shape[1] == 1:
    X = np.insert(X, 1, 1, axis=1)
print(np.shape(X))

In [ ]:
#b)
y_shifted = y+1e6

#No Bias Part
b_arr = [np.array(b1) for b1 in np.linspace(-1000, 1000, num=2001)]
cost_arr = [cost(predict(b, X[:,0]), y_shifted) for b in b_arr]
best_b1 = b_arr[np.argmin(cost_arr)]


#Bias Part
nb_of_bs = 101 # compute the cost nb_of_bs times in each dimension
b1 = np.linspace(-500, 2000, num=nb_of_bs) # slope coefficient
b2 = np.linspace(-1e5, 3e6, num=nb_of_bs) # bias
b_x, b_y = np.meshgrid(b1, b2) # generate grid
cost_arr_2d = np.zeros((nb_of_bs, nb_of_bs)) # initialize cost matrix

# Fill the cost matrix for each combination of coefficients
for i in range(nb_of_bs):
    for j in range(nb_of_bs):
        cost_arr_2d[i,j] = cost(predict(np.array([[b_x[i,j]], [b_y[i,j]]]), X) , y_shifted)


plt.figure(figsize=(15,20)) 
        
plt.subplot(2,1,1)
plt.title("Cost heatmap for bias and slope")

plt.contourf(b_x, b_y, np.log(cost_arr_2d), 20, alpha=0.9, cmap=cm.pink)
cbar = plt.colorbar()
plt.scatter(best_b1, 0, label = "Best solution without bias")
cbar.ax.set_ylabel('log(cost)')
plt.xlabel("b1 (slope)")
plt.ylabel("b2 (bias)")
plt.legend()

plt.subplot(2,1,2)

plt.scatter(X.T[0], y_shifted)
plt.xlabel("living sqft")
plt.ylabel("price")

x_model = np.linspace(np.min(X), np.max(X), 1000)
y_model = best_b1*x_model
plt.plot(x_model, y_model, label='No Bias. Best b1 = {}, error = {:.2E}'.format(best_b1, cost_arr[np.argmin(cost_arr)]))

best_b1_2d_ind, best_b2_2d_ind = np.unravel_index(cost_arr_2d.argmin(), cost_arr_2d.shape)
best_b1 = b_x[best_b1_2d_ind,best_b2_2d_ind]
best_b2 = b_y[best_b1_2d_ind,best_b2_2d_ind]

y_model = best_b1*x_model + best_b2
plt.plot(x_model, y_model, label='Best b1 = {}, b2 = {}, error = {:.2E}'.format(best_b1, best_b2, cost_arr_2d[best_b1_2d_ind,best_b2_2d_ind]))

plt.legend()

plt.show()

**Adding bias improved error by three orders of magnitude. Not bad!**

# 3. Standardization

In order to improve performance of many machine learning algorithms, the value standardization is applied. It makes the values of each feature in the data have zero-mean and unit-variance. It is achieved by substracting from each value $x$ the overall mean $\bar{x}$, and then dividing it by feature's standard deviation ${\sigma}$:

$$x' = \frac{x - \bar{x}}{\sigma}. $$

It is important to remember, that bias column should not be standardize!

## Exercise 3
a) Implement function standardize(), which standardize the feature matrix and returns it together with two vectors containing original features' means and standard deviations.  
b) Implement function destandardize(), which restores the original data given standardized feature matrix X and vectors containing original features' means and standard deviations.  
c) Plot original $X,y$ data. Then plot it after standardization. Then plot it after destandardisation.

In [ ]:
#a)
def standardize(X):
    X_std = np.std(X, axis=0)
    X_mean = np.mean(X, axis=0)
    return (X - X_mean) / X_std, X_mean, X_std

#b)
def destandardize(X, means, sds):
    return X * sds + means


#short test:
W = np.array([[1,2,3],[2,5,4],[3,4,5],[4,5,7]])
W2, W_mean, W_std = standardize(W)
print(W2)
print(W_mean)
print(W_std)
print(destandardize(W2, W_mean, W_std))

#Hint: with np.mean and np.std first function is three lines long. The second is one-liner.

In [ ]:
#c)

if X.shape[1] == 2:
    X = np.delete(X, 1, axis=1)

X2, X_mean, X_std = standardize(X)
y2, y_mean, y_std = standardize(y)

plt.scatter(X, y)
plt.show()

plt.scatter(X2, y2)
plt.show()

plt.scatter(destandardize(X2, X_mean, X_std), destandardize(y2, y_mean, y_std))
plt.show()


## 3. Gradient Descent Method

In previous examples we were determining the approximate value of optimal vector **$b$** by finding best arguments from pre-defined grid. This solution is neither fast nor precise. Although in theory it is possible to find vector **$b$** analytically, it requires inversing large matrices, as the close form for **$b$** is $b=(X^\mathsf{T}X)^{-1}X^\mathsf{T}y$.

The other approach is to find approximation of **$b$** is Gradient Descent Method. Let us recall that for function $f: \ R^n \to R$ a gradient 

$$\nabla f =  \begin{bmatrix} \frac{\partial f}{\partial x_1} \\ \vdots \\ \frac{\partial f}{\partial x_n} \end{bmatrix}$$

is the direction of the biggest increase of $f$. Using linearity of derivative, $-\nabla f$ is the direction of the biggest decrease. Gradient Descent Method is based on iterative moving in the direction opposite to gradient, and by these means approaching the minimum.

#### GDM step by step:  
1. Choose starting point $x_{0}$ and parameters: *learning rate* and *precision*,
2. Update $x_{i+1} = x_{i} - \nabla f(x_{i})\cdot lr$, where $lr$ is learning rate parameter,
3. If $(|\nabla f(x_{i})| < precision)$, end. If not, go back to point 2.

One of many limitations of GMD may be knowledge of function's gradient. Luckily for us, it is quite easy in case of linear regression with mean square error cost function. We have

$$ \nabla f_{y}(b) = \nabla\dfrac{|Xb−y|^2}{n} = \dfrac{2X^\mathsf{T}(Xb−y)}{n}.  $$

## Exercise 4
a) Create functions: *gradient()* which computes gradient for linear regresion and *gradient_descent_step()* which returns new vector **b** being the result of one GDM step.  

b) Take **$X$** consisting of *sqft_living* together with bias column. Set **$y$** as price. Standardize both **$X$** (without bias) and **$y$**. Plot a heatmap showing dependence of cost function's value on vector **$b$**. Mark first 5 steps of gradient descent with *learning rate* = $0.2$, starting from **$b =  \begin{bmatrix} 3 \\ 2 \end{bmatrix}$**. Experiment with other *learning rates*.  

c) Take **$X$** consisting of *sqft_living* and *sqft_lot*. Set **$y$** as price. Standardize both **$X$** and **$y$**. Plot a heatmap showing dependence of cost function's value on vector **$b$**. Mark first 5 steps of gradient descent with *learning rate* = $0.2$, starting from **$b =  \begin{bmatrix} 3 \\ 1 \end{bmatrix}$**. Experiment with other *learning rates*.  

d) Try to redo above points without standarization. How can you explain such results?


In [ ]:
#a)

def gradient(b, X, y):
    return 2 * X.T.dot(X.dot(b) - y) / len(X)

def gradient_descent_step(b, X, y, lr):
    return b - gradient(b, X, y).dot(lr)


#short test:
test_b = np.array([[1],[1],[1]])
test_X = np.array([[1,2,3],[2,5,4],[3,4,5],[4,5,7]])
test_y = np.array([[1],[2],[3],[4]])

print(gradient(test_b, test_X, test_y))
print(gradient_descent_step(test_b, test_X, test_y, 0.1))

In [ ]:
#b)
X = house_df.sqft_living.values.reshape(-1,1)[:]
y = house_df.price.values.reshape(-1,1)[:]

X2, X_mean, X_std = standardize(X)
y2, y_mean, y_std = standardize(y)

X2 = np.insert(X2, 1, 1, axis=1)

# Plot a heatmap showing dependence of cost function's value on vector b.
nb_of_bs = 101 # compute the cost nb_of_bs times in each dimension
b1 = np.linspace(-4, 4, num=nb_of_bs) # slope coefficient
b2 = np.linspace(-4, 4, num=nb_of_bs) # bias
b_x, b_y = np.meshgrid(b1, b2) # generate grid
cost_arr_2d = np.zeros((nb_of_bs, nb_of_bs)) # initialize cost matrix

# Fill the cost matrix for each combination of coefficients
for i in range(nb_of_bs):
    for j in range(nb_of_bs):
        cost_arr_2d[i,j] = cost(
            predict(
                np.array(
                    [
                        [b_x[i, j]],
                        [b_y[i, j]]
                    ]
                ),
                X2
            ),
            y2
        )

plt.title("Cost heatmap for bias and slope")
plt.contourf(
    b_x,
    b_y,
    np.log(cost_arr_2d),
    20,
    #alpha=0.9,
    cmap=cm.pink,
)
cbar = plt.colorbar()
cbar.ax.set_ylabel('log(cost)')
plt.xlabel("b1 (slope)")
plt.ylabel("b2 (bias)")

b = [np.array([[3], [2]])] # np.array jest niepotrzebne, ale powinno szybciej działać
rate = 0.2

for i in range(5):
    b.append(gradient_descent_step(b[-1], X2, y2, rate))
plt.plot(*zip(*b), marker='.') # *zip, bo potrzebujemy 2 współrzędne rozbić na 2 tablice

# etykiety
for i in range(len(b)):
    plt.text(b[i][0] - 0.05, b[i][1] - 0.2, f'w({i})', color='blue', fontsize=7, rotation=-55)

plt.show()

# Mark first 5 steps of gradient descent with learning rate = 0.2, starting from b=[32].
# Experiment with other learning rates. 

In [ ]:
#c)
X = house_df[['sqft_living', 'sqft_lot']].values.reshape(-1,2)[:]
y = house_df.price.values.reshape(-1,1)[:]

X2, X_mean, X_std = standardize(X)
y2, y_mean, y_std = standardize(y)

# Plot a heatmap showing dependence of cost function's value on vector b.
nb_of_bs = 101 # compute the cost nb_of_bs times in each dimension
b1 = np.linspace(-5, 5, num=nb_of_bs) # slope coefficient
b2 = np.linspace(-4, 4, num=nb_of_bs) # bias
b_x, b_y = np.meshgrid(b1, b2) # generate grid
cost_arr_2d = np.zeros((nb_of_bs, nb_of_bs)) # initialize cost matrix

# Fill the cost matrix for each combination of coefficients
for i in range(nb_of_bs):
    for j in range(nb_of_bs):
        cost_arr_2d[i,j] = cost(
            predict(
                np.array(
                    [
                        [b_x[i, j]],
                        [b_y[i, j]]
                    ]
                ),
                X2
            ),
            y2
        )

plt.contourf(
    b_x,
    b_y,
    np.log(cost_arr_2d),
    20,
    #alpha=0.9,
    cmap=cm.pink,
)
cbar = plt.colorbar()
cbar.ax.set_ylabel('log(cost)')
plt.xlabel("sqft_living")
plt.ylabel("sqft_lot")

b = [np.array([[3], [1]])] # np.array jest niepotrzebne, ale powinno szybciej działać
rate = 0.2

for i in range(5):
    b.append(gradient_descent_step(b[-1], X2, y2, rate))
plt.plot(*zip(*b), marker='.') # *zip, bo potrzebujemy 2 współrzędne rozbić na 2 tablice

# etykiety
for i in range(len(b)):
    plt.text(b[i][0] - 0.05, b[i][1] - 0.2, f'w({i})', color='blue', fontsize=7, rotation=-55)

plt.show()

In [ ]:
#d)
X = house_df.sqft_living.values.reshape(-1,1)[:]
y = house_df.price.values.reshape(-1,1)[:]

X2 = X # X2, X_mean, X_std = standardize(X)
y2 = y # y2, y_mean, y_std = standardize(y)

X2 = np.insert(X2, 1, 1, axis=1)

# Plot a heatmap showing dependence of cost function's value on vector b.
nb_of_bs = 101 # compute the cost nb_of_bs times in each dimension
b1 = np.linspace(-4, 4, num=nb_of_bs) # slope coefficient
b2 = np.linspace(-4, 4, num=nb_of_bs) # bias
b_x, b_y = np.meshgrid(b1, b2) # generate grid
cost_arr_2d = np.zeros((nb_of_bs, nb_of_bs)) # initialize cost matrix

# Fill the cost matrix for each combination of coefficients
for i in range(nb_of_bs):
    for j in range(nb_of_bs):
        cost_arr_2d[i,j] = cost(
            predict(
                np.array(
                    [
                        [b_x[i, j]],
                        [b_y[i, j]]
                    ]
                ),
                X2
            ),
            y2
        )

plt.title("Cost heatmap for bias and slope")
plt.contourf(
    b_x,
    b_y,
    np.log(cost_arr_2d),
    20,
    #alpha=0.9,
    cmap=cm.pink,
)
cbar = plt.colorbar()
cbar.ax.set_ylabel('log(cost)')
plt.xlabel("b1 (slope)")
plt.ylabel("b2 (bias)")

b = [np.array([[3], [2]])] # np.array jest niepotrzebne, ale powinno szybciej działać
rate = 0.2

for i in range(5):
    b.append(gradient_descent_step(b[-1], X2, y2, rate))
plt.plot(*zip(*b), marker='.') # *zip, bo potrzebujemy 2 współrzędne rozbić na 2 tablice

# etykiety
for i in range(len(b)):
    plt.text(b[i][0] - 0.05, b[i][1] - 0.2, f'w({i})', color='blue', fontsize=7, rotation=-55)

plt.show()

In [ ]:
X = house_df[['sqft_living', 'sqft_lot']].values.reshape(-1,2)[:]
y = house_df.price.values.reshape(-1,1)[:]

X2 = X # X2, X_mean, X_std = standardize(X)
y2 = y # y2, y_mean, y_std = standardize(y)

# Plot a heatmap showing dependence of cost function's value on vector b.
nb_of_bs = 101 # compute the cost nb_of_bs times in each dimension
b1 = np.linspace(-4, 4, num=nb_of_bs) # slope coefficient
b2 = np.linspace(-4, 4, num=nb_of_bs) # bias
b_x, b_y = np.meshgrid(b1, b2) # generate grid
cost_arr_2d = np.zeros((nb_of_bs, nb_of_bs)) # initialize cost matrix

# Fill the cost matrix for each combination of coefficients
for i in range(nb_of_bs):
    for j in range(nb_of_bs):
        cost_arr_2d[i,j] = cost(
            predict(
                np.array(
                    [
                        [b_x[i, j]],
                        [b_y[i, j]]
                    ]
                ),
                X2
            ),
            y2
        )

plt.title("Cost heatmap for bias and slope")
plt.contourf(
    b_x,
    b_y,
    np.log(cost_arr_2d),
    20,
    #alpha=0.9,
    cmap=cm.pink,
)
cbar = plt.colorbar()
cbar.ax.set_ylabel('log(cost)')
plt.xlabel("sqft_living")
plt.ylabel("sqft_lot")

b = [np.array([[3], [1]])] # np.array jest niepotrzebne, ale powinno szybciej działać
rate = 0.2

for i in range(5):
    b.append(gradient_descent_step(b[-1], X2, y2, rate))
plt.plot(*zip(*b), marker='.') # *zip, bo potrzebujemy 2 współrzędne rozbić na 2 tablice

# etykiety
for i in range(len(b)):
    plt.text(b[i][0] - 0.05, b[i][1] - 0.2, f'w({i})', color='blue', fontsize=7, rotation=-55)

plt.show()